In [9]:
import requests, sys, time
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
url = 'localhost'

In [13]:
driver_install = ChromeDriverManager().install()

In [14]:
service = Service(driver_install)
driver = (service)